#Single-fidelity GP surrogate model

In the following notebook construct, train, and validate a single-fidelity Gaussian Process (GP) surrogate model of lung function.

We begin by importing the necessary modules.

In [ ]:
from IPython.display import clear_output

In [ ]:
# Install Emukit (useful for the construction of our surrogate model)
!pip install emukit
clear_output()
print('Emukit installed')

Emukit installed


In [ ]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
%matplotlib inline

# Emukit imports
import GPy
import emukit.multi_fidelity
import emukit.test_functions
import emukit.test_functions.multi_fidelity
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel

np.random.seed(20)

## Import high-fidelity dataset

Next, we import and load the created dataset from high fidelity simulations. Each sample/observations of the dataset consists in an input and output array of the form:

\begin{array}{cc}
\text{Input} & \text{Output}\\
[ c, \beta, c_1, c_3, k, K_s ] & [ \mathrm{C_{rs}}, \mathrm{R} ]
\end{array}
\
,where the input contains our six model parameters ($c$, $\beta$, $c_1$ and $c_3$ from the tissue constitutive model + permeability $k$ + chest wall stiffness $K_s$) and the output our two variables that represent the lung response (respiratory system compliance $\mathrm{C_{rs}}$ and airways resistance $\mathrm{R}$).

\
- For the surrogate model training, we have 20 samples.

- For the surrogate model testing, we have 100 samples.

In [ ]:
# Getting files from repository
!wget https://raw.githubusercontent.com/josebarahonay/datos/master/training_input_data_20_hf.npy
!wget https://raw.githubusercontent.com/josebarahonay/datos/master/training_output_data_20_hf.npy
!wget https://raw.githubusercontent.com/josebarahonay/datos/master/testing_input_data_100_hf.npy
!wget https://raw.githubusercontent.com/josebarahonay/datos/master/testing_output_data_100_hf.npy

clear_output()

In [ ]:
# High Fidelity training datasets
X_hf = np.load('training_input_data_20_hf.npy')
Y_hf = np.load('training_output_data_20_hf.npy')

# High Fidelity testing datasets
X_hf_100 = np.load('testing_input_data_100_hf.npy')
Y_hf_100 = np.load('testing_output_data_100_hf.npy')

print("Checking size of HF and LF datasets:")
print("")
print("Training X HF size :", X_hf.shape)
print("Training Y HF size :", Y_hf.shape)
print("")
print("Testing X HF size :", X_hf_100.shape)
print("Testing Y HF size :", Y_hf_100.shape)

Checking size of HF and LF datasets:

Training X HF size : (20, 6)
Training Y HF size : (20, 2)

Testing X HF size : (100, 6)
Testing Y HF size : (100, 2)


## Data preprocessing for single-fidelity Gaussian Process (GP) model

We divide our training dataset into an internal training and validation sets. As there are only a few observations, we want to use the majority of them for training our surrogate model. Thus, we specify a training size of 95%, leaving only one observation for the internal model validation.The testing dataset remains untouched for the entire analysis and is only used to evaluate the performance after we trained our surrogate model.

In [ ]:
from sklearn.model_selection import train_test_split
# Next, the High Fidelity training datasets is split into an internal Training and Validation set
X_hf_train, X_hf_test = train_test_split(X_hf, train_size=0.95, shuffle=True, random_state=1) # this split the input data
Y_hf_train, Y_hf_test = train_test_split(Y_hf, train_size=0.95, shuffle=True, random_state=1) # this split the output data

X_hf_test_100 = X_hf_100
Y_hf_test_100 = Y_hf_100

Given that the six model parameters have considerable differences in their scale, it is necessary to normalize the input dataset:

In [ ]:
from sklearn.preprocessing import StandardScaler

## Normalize input data
scaler = StandardScaler()

X_hf_train = scaler.fit_transform(X_hf_train)
X_hf_test = scaler.transform(X_hf_test)
X_hf_test_100 = scaler.transform(X_hf_test_100)

## ML surrogate model construction

After the data is prepared, we construct and train the single-fidelity GP model.

Regarding this, we want to learn a latent function $f$ from our dataset, such that $\boldsymbol{y}=f(\boldsymbol{X})+\epsilon$.

First, we establish that our prior knowledge about $f$ can be described by a GP with mean $m(\boldsymbol{x})$ and covariance $k(\boldsymbol{x}, \boldsymbol{x}^{\prime} ; \boldsymbol{\theta})$,

\begin{equation}
    f(\boldsymbol{x}) \sim \mathcal{G} \mathcal{P}\left(m(\boldsymbol{x}), k(\boldsymbol{x}, \boldsymbol{x}^{\prime} ; \boldsymbol{\theta})\right),
\end{equation}

where we assume a zero-mean function $m(\boldsymbol{x}) = 0$. In our prior, $\boldsymbol{\theta}$ denotes the hyperparameters of the covariance function or kernel, which encodes our assumption about the Gaussian process. Here, we choose a quadratic exponential kernel (RBF), expressed in the form

\begin{equation}
    k(\boldsymbol{x},\boldsymbol{x}';\boldsymbol{\theta})=v\exp\left\{-\sum\limits_{i=1}^d\dfrac{\left(x_i-x_i'\right)^2}{2\ell_i^2}\right\},       
\end{equation}

where $v$ y $\{\ell_i\}_{i=1,\ldots,d}$ are the model hyperparameters: $v$ models the variance of the process, and $\ell_i$ determines the lengthscale of each input dimension.

After training the model, we compute the prediction of the posterior distribution,

\begin{equation}
    p(\boldsymbol{y}^{*}|\boldsymbol{x}^{*},\mathcal{D})\sim\mathcal{N}(\mu(\boldsymbol{x}^{*}),\varSigma(\boldsymbol{x}^{*}))
\end{equation}

\
From here, it is important to note that we build two separate models: one for the respiratory-system compliance prediction, and the other for the airways resistance.

In [ ]:
input_dim = X_hf_train.shape[1]
Y_hf_train_r = Y_hf_train[:,0][:,np.newaxis]
Y_hf_train_c = Y_hf_train[:,1][:,np.newaxis]

## Construct a single-fidelity model
kernel_r = GPy.kern.MLP(input_dim, ARD = False)
kernel_c = GPy.kern.MLP(input_dim, ARD = False)

### RESISTANCE model ###
m_r = GPy.models.GPRegression(X_hf_train, Y_hf_train_r, kernel_r)
### COMPLIANCE model ###
m_c = GPy.models.GPRegression(X_hf_train, Y_hf_train_c, kernel_c)

## Fit (optimize) the models
#m.optimize(optimizer='scg', max_iters=1000)
m_r.optimize(optimizer='scg', max_iters=10000)
print('Resistance model optimized')
m_c.optimize(optimizer='scg', max_iters=10000)
print('Compliance model optimized')

Resistance model optimized
Compliance model optimized


After trained, we evaluate the model performance and save the model predictions on the HF testing set

In [ ]:
# Predictions
Y_pred_mean_R, Y_pred_var_R = m_r.predict(X_hf_test_100)
Y_pred_mean_C, Y_pred_var_C = m_c.predict(X_hf_test_100)

# Performance
print(r2_score(Y_hf_test_100[:,0], Y_pred_mean_R[:,0]))
print(r2_score(Y_hf_test_100[:,1], Y_pred_mean_C[:,0]))

# Export values
np.save('gp_var_C_50percent.npy', Y_pred_var_C)
np.save('gp_var_R_50percent.npy', Y_pred_var_R)
np.save('gp_C_50percent.npy', Y_pred_mean_C)
np.save('gp_R_50percent.npy', Y_pred_mean_R)

0.9012799314108828
0.9057251926827575


## Cross-Validation (in testing set)

Additionally, we perform a cross-validation using different training sizes.

**Note**: *Take consideration that the following code block may take several hours to be executed*.

In [ ]:
split_list = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
random_list = np.arange(1, 100+1,10)
RMSE_list_R = []
RMSE_list_C = []
std_R = []
std_C = []

for split in split_list:
  print("Train split: ", split)
  RMSE_inner_list_R = []
  RMSE_inner_list_C = []
  for random_number in random_list:
    # Split
    X_hf_train, X_hf_test = train_test_split(X_hf, train_size=split, shuffle=True, random_state=random_number) # this split the input data
    Y_hf_train, Y_hf_test = train_test_split(Y_hf, train_size=split, shuffle=True, random_state=random_number) # this split the output data

    X_hf_test_100 = X_hf_100
    Y_hf_test_100 = Y_hf_100

    # Normalize
    scaler = StandardScaler()

    X_hf_train = scaler.fit_transform(X_hf_train)
    X_hf_test = scaler.transform(X_hf_test)
    X_hf_test_100 = scaler.transform(X_hf_test_100)

    # Train
    input_dim = X_hf_train.shape[1]
    Y_hf_train_r = Y_hf_train[:,0][:,np.newaxis]
    Y_hf_train_c = Y_hf_train[:,1][:,np.newaxis]

    ## Construct a single-fidelity model
    kernel_r = GPy.kern.MLP(input_dim, ARD = False)
    kernel_c = GPy.kern.MLP(input_dim, ARD = False)
    m_r = GPy.models.GPRegression(X_hf_train, Y_hf_train_r, kernel_r)
    m_c = GPy.models.GPRegression(X_hf_train, Y_hf_train_c, kernel_c)
    #print(m)
    #m.optimize(optimizer='scg', max_iters=1000)
    m_r.optimize(optimizer='scg', max_iters=10000)
    m_c.optimize(optimizer='scg', max_iters=10000)
    #print(m)

    # Evaluate
    Y_pred_mean_r, Y_pred_var_r = m_r.predict(X_hf_test_100)
    Y_pred_mean_c, Y_pred_var_c = m_c.predict(X_hf_test_100)
    # H-F predictions
    print("H-F predictions")
    mse_R = mean_squared_error(Y_hf_test_100[:,0], Y_pred_mean_r[:,0])
    print('\nMSE R: {}'.format(mse_R))
    mse_C = mean_squared_error(Y_hf_test_100[:,1], Y_pred_mean_c[:,0])
    print('\nMSE C: {}'.format(mse_C))
    RMSE_inner_list_R.append(np.sqrt(mse_R))
    RMSE_inner_list_C.append(np.sqrt(mse_C))
  RMSE_list_R.append(np.average(RMSE_inner_list_R))
  RMSE_list_C.append(np.average(RMSE_inner_list_C))
  print(RMSE_inner_list_R)
  print(RMSE_inner_list_C)
  std_R.append(np.std(RMSE_inner_list_R))
  std_C.append(np.std(RMSE_inner_list_C))

#clear_output()

for i in range(len(split_list)):
  print("Train split: ", split_list[i], "||    MSE R: ", RMSE_list_R[i], "||    MSE C: ", RMSE_list_C[i], "||    std R: ", std_R[i], "||    std C: ", std_C[i])

Train split:  0.5
H-F predictions

MSE R: 0.2730843250091638

MSE C: 688.1293647850471
H-F predictions

MSE R: 0.3323139445850639

MSE C: 238.19728541243703
H-F predictions

MSE R: 0.48008492827275634

MSE C: 362.7645151415117
H-F predictions

MSE R: 0.19763010688421953

MSE C: 181.47332179169655
H-F predictions

MSE R: 0.3410461200682547

MSE C: 565.3855067588119
H-F predictions

MSE R: 0.4114905090358284

MSE C: 403.54090180124126
H-F predictions

MSE R: 0.49988082521487015

MSE C: 312.87852654268943
H-F predictions

MSE R: 0.23056348020696127

MSE C: 145.77481330120608
H-F predictions

MSE R: 0.310287079214099

MSE C: 554.8226811923288
H-F predictions

MSE R: 0.2033182837413549

MSE C: 460.2443960910952
[0.5225747075865458, 0.5764667766533158, 0.6928816120180679, 0.44455607844704986, 0.5839915410930664, 0.6414752598782189, 0.7070225068658494, 0.4801702616853331, 0.5570341813695987, 0.45090828750573536]
[26.232219974394983, 15.433641352980736, 19.046378005844357, 13.471203427745294, 

In [ ]:
# Export values
np.save('RMSE_R_sfgp_updated.npy', RMSE_list_R)
np.save('RMSE_C_sfgp_updated.npy', RMSE_list_C)
np.save('std_R_sfgp_updated.npy', std_R)
np.save('std_C_sfgp_updated.npy', std_C)